In [137]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import numpy.random as nr
import math
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn import linear_model, svm
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
import sklearn.metrics as sklm
from dateutil.relativedelta import relativedelta

%matplotlib inline

In [122]:
work_cust = pd.read_csv('work_cust.csv',index_col='CustomerID')
work_cust.drop(['Unnamed: 0','dummy','AddressLine1','FirstName','LastName','StateProvinceName',
                'PhoneNumber','BirthDate','AveMonthSpend','PostalCode'], axis = 1, inplace = True)
work_cust.head().transpose()

CustomerID                   11000         11001         11002         11003  \
City                   Rockhampton       Seaford        Hobart    North Ryde   
CountryRegionName        Australia     Australia     Australia     Australia   
Education               Bachelors     Bachelors     Bachelors     Bachelors    
Occupation            Professional  Professional  Professional  Professional   
Gender                           M             M             M             F   
MaritalStatus                    M             S             M             S   
HomeOwnerFlag                    1             0             1             0   
NumberCarsOwned                  0             1             1             1   
NumberChildrenAtHome             0             3             3             0   
TotalChildren                    2             3             3             0   
YearlyIncome                137947        101141         91945         86688   
BikeBuyer                        0             1             0             0   
Age                             52            53            53            50   

CustomerID                   11004  
City                    Wollongong  
CountryRegionName        Australia  
Education               Bachelors   
Occupation            Professional  
Gender                           F  
MaritalStatus                    S  
HomeOwnerFlag                    1  
NumberCarsOwned                  4  
NumberChildrenAtHome             5  
TotalChildren                    5  
YearlyIncome                 92771  
BikeBuyer                        1  
Age                             50

In [44]:
print(work_cust.HomeOwnerFlag.value_counts())

1    11058
0     5346
Name: HomeOwnerFlag, dtype: int64


In [41]:
work_cust.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16404 entries, 11000 to 18982
Data columns (total 12 columns):
CountryRegionName       16404 non-null object
Education               16404 non-null object
Occupation              16404 non-null object
Gender                  16404 non-null object
MaritalStatus           16404 non-null object
HomeOwnerFlag           16404 non-null int64
NumberCarsOwned         16404 non-null int64
NumberChildrenAtHome    16404 non-null int64
TotalChildren           16404 non-null int64
YearlyIncome            16404 non-null int64
BikeBuyer               16404 non-null int64
Age                     16404 non-null int64
dtypes: int64(7), object(5)
memory usage: 1.6+ MB


In [73]:
# Label
labels = np.array(work_cust['BikeBuyer'])
labels.shape

(16404,)

In [85]:
# Encode categorical data
def encode_string(cat_features):
    
    ## First encode the strings to numeric categories
    enc = preprocessing.LabelEncoder()
    enc.fit(cat_features)
    enc_cat_features = enc.transform(cat_features)
    
    ## Now, apply one hot encoding
    ohe = preprocessing.OneHotEncoder()
    encoded = ohe.fit(enc_cat_features.reshape(-1,1))
    return encoded.transform(enc_cat_features.reshape(-1,1)).toarray()

categorical_columns = ['Education','Occupation',
                       'Gender','MaritalStatus','HomeOwnerFlag']

Features = encode_string(work_cust['CountryRegionName'])

for col in categorical_columns:
    temp = encode_string(work_cust[col])
    Features = np.concatenate([Features, temp], axis = 1)
    
print(Features.shape)

(16404, 22)


In [86]:
# Add numerical feature
Features = np.concatenate([Features, np.array(work_cust[['NumberCarsOwned',
                                                        'NumberChildrenAtHome',
                                                        'TotalChildren',
                                                        'YearlyIncome','Age']])], axis = 1)

print(Features.shape)

(16404, 27)


The code in the cell below performs the following processing:
1. An index vector is Bernoulli sampled using the `train_test_split` function from the `model_selection` package of scikit-learn. 
2. The first column of the resulting index array contains the indices of the samples for the training cases. 
3. The second column of the resulting index array contains the indices of the samples for the test cases. 

In [87]:
## Randomly sample cases to create independent training and test data
nr.seed(9988)
indx = range(Features.shape[0])
indx = ms.train_test_split(indx, test_size = 0.3)
X_train = Features[indx[0],:]
y_train = np.ravel(labels[indx[0]])
X_test = Features[indx[1],:]
y_test = np.ravel(labels[indx[1]])

In [88]:
scaler = preprocessing.StandardScaler().fit(X_train[:,23:])
X_train[:,23:] = scaler.transform(X_train[:,23:])
X_test[:,23:] = scaler.transform(X_test[:,23:])

#### Construct the logistic regression model

In [118]:
#NB_mod = GaussianNB()
nn_mod = MLPClassifier(hidden_layer_sizes = (50,))
#logistic_mod = linear_model.LogisticRegression()
svm_mod = svm.LinearSVC()

nn_mod.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(50,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [121]:
def print_metrics(labels, scores):
    metrics = sklm.precision_recall_fscore_support(labels, scores)
    conf = sklm.confusion_matrix(labels, scores)
    print('                 Confusion matrix')
    print('                 Score positive    Score negative')
    print('Actual positive    %6d' % conf[0,0] + '             %5d' % conf[0,1])
    print('Actual negative    %6d' % conf[1,0] + '             %5d' % conf[1,1])
    print('')
    print('Accuracy  %0.2f' % sklm.accuracy_score(labels, scores))
    print(' ')
    print('           Positive      Negative')
    print('Num case   %6d' % metrics[3][0] + '        %6d' % metrics[3][1])
    print('Precision  %6.2f' % metrics[0][0] + '        %6.2f' % metrics[0][1])
    print('Recall     %6.2f' % metrics[1][0] + '        %6.2f' % metrics[1][1])
    print('F1         %6.2f' % metrics[2][0] + '        %6.2f' % metrics[2][1])


    
print_metrics(y_test, scores)

                 Confusion matrix
                 Score positive    Score negative
Actual positive      2908               396
Actual negative       620               998

Accuracy  0.79
 
           Positive      Negative
Num case     3304          1618
Precision    0.82          0.72
Recall       0.88          0.62
F1           0.85          0.66


### Test using new dataset

In [162]:
ads_test = pd.read_csv('./FinalExam-Test/AW_test.csv')
ads_test.head()

CustomerID Title FirstName MiddleName  LastName Suffix  \
0       18988   NaN  Courtney          A     Baker    NaN   
1       29135   NaN      Adam          C     Allen    NaN   
2       12156   NaN    Bonnie        NaN      Raji    NaN   
3       13749   NaN     Julio          C    Alonso    NaN   
4       27780   NaN   Christy          A  Andersen    NaN   

             AddressLine1 AddressLine2        City StateProvinceName  \
0   8727 Buena Vista Ave.          NaN     Fremont        California   
1        3491 Cook Street          NaN       Haney  British Columbia   
2    359 Pleasant Hill Rd          NaN     Burbank        California   
3        8945 Euclid Ave.          NaN  Burlingame        California   
4  42, boulevard Tremblay          NaN   Dunkerque              Nord   

       ...       BirthDate        Education      Occupation Gender  \
0      ...        1/5/1945       Bachelors       Management      F   
1      ...       10/4/1964       Bachelors   Skilled Manual      M   
2      ...       1/12/1934  Graduate Degree      Management      F   
3      ...       9/22/1958  Graduate Degree  Skilled Manual      M   
4      ...       3/19/1965      High School          Manual      F   

  MaritalStatus HomeOwnerFlag NumberCarsOwned NumberChildrenAtHome  \
0             S             0               2                    0   
1             M             1               2                    2   
2             M             1               2                    0   
3             M             1               0                    0   
4             M             1               1                    2   

   TotalChildren  YearlyIncome  
0              5         86931  
1              4        100125  
2              4        103985  
3              4        127161  
4              2         21876  

[5 rows x 23 columns]

In [140]:
# Calculate age of customer based on birthdate
def calculate_age(end):
    r = relativedelta(pd.to_datetime('now'), pd.to_datetime(end)) 
    return '{}'.format(r.years)


ads_test['Age'] = ads_test['BirthDate'].apply(calculate_age)

# Convert age to integer
ads_test['Age'] = ads_test['Age'].astype('int64')

In [143]:
ads_totest = ads_test[['CountryRegionName','Education','Occupation','Gender',
                      'MaritalStatus','HomeOwnerFlag','NumberCarsOwned',
                      'NumberChildrenAtHome','TotalChildren','YearlyIncome',
                      'Age']]

ads_totest.head()

CountryRegionName        Education      Occupation Gender  \
CustomerID                                                             
18988          United States       Bachelors       Management      F   
29135                 Canada       Bachelors   Skilled Manual      M   
12156          United States  Graduate Degree      Management      F   
13749          United States  Graduate Degree  Skilled Manual      M   
27780                 France      High School          Manual      F   

           MaritalStatus  HomeOwnerFlag  NumberCarsOwned  \
CustomerID                                                 
18988                  S              0                2   
29135                  M              1                2   
12156                  M              1                2   
13749                  M              1                0   
27780                  M              1                1   

            NumberChildrenAtHome  TotalChildren  YearlyIncome  Age  
CustomerID                                                          
18988                          0              5         86931   73  
29135                          2              4        100125   54  
12156                          0              4        103985   84  
13749                          0              4        127161   60  
27780                          2              2         21876   53

In [144]:
# Encode categorical data
def encode_string(cat_features):
    
    ## First encode the strings to numeric categories
    enc = preprocessing.LabelEncoder()
    enc.fit(cat_features)
    enc_cat_features = enc.transform(cat_features)
    
    ## Now, apply one hot encoding
    ohe = preprocessing.OneHotEncoder()
    encoded = ohe.fit(enc_cat_features.reshape(-1,1))
    return encoded.transform(enc_cat_features.reshape(-1,1)).toarray()

categorical_columns = ['Education','Occupation',
                       'Gender','MaritalStatus','HomeOwnerFlag']

Features = encode_string(ads_totest['CountryRegionName'])

for col in categorical_columns:
    temp = encode_string(ads_totest[col])
    Features = np.concatenate([Features, temp], axis = 1)
    
print(Features.shape)

(500, 22)


In [145]:
# Add numerical feature
Features = np.concatenate([Features, np.array(ads_totest[['NumberCarsOwned',
                                                        'NumberChildrenAtHome',
                                                        'TotalChildren',
                                                        'YearlyIncome','Age']])], axis = 1)

print(Features.shape)

(500, 27)


In [146]:
X_test = Features
X_test[:,23:] = scaler.transform(X_test[:,23:])

In [155]:
scores = nn_mod.predict(X_test)
print(scores)

[0 1 0 0 0 0 1 1 1 0 1 0 0 0 0 0 0 0 1 0 0 1 1 1 0 0 0 1 1 0 1 0 0 0 1 1 1
 0 0 0 0 1 0 0 0 0 0 1 1 1 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 1 1 1 0 1 0 0 0 0
 0 1 1 0 0 0 1 1 0 1 0 0 1 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1
 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 1 0 0 0 1 0 0 1 1 0 1 0 0 0 0 0 1 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 0 0 1 0 0 1 0 0 1 0 0
 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 1 0 0 1 0 1 0 0 0 0 0 1 1 1 0
 0 1 0 0 0 1 1 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1
 0 0 0 1 0 1 1 0 1 1 0 1 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1
 1 0 0 0 0 0 0 1 0 0 1 1 1 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 1
 0 1 1 1 0 1 0 0 1 0 1 0 0 1 0 1 0 0 1 0 1 0 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 1 0 0 0 0 0 1 0 0 1 1 0 0 0 0 1 1 1 1 0 1
 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 1 0 0 0 1 0 0
 0 0 0 1 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 1 1 1
 1 1 0 0 0 0 0 0 0 1 1 0 

In [177]:
result = pd.DataFrame(scores, index = ads_test['CustomerID'],columns=['BikeBuyer'])

result
result.to_csv('result.csv')